In [1]:
import numpy as np
import tensorflow as tf # gives warning from h5py but it is fine.
from matplotlib import pyplot as plt
%matplotlib inline
from IPython.display import display, clear_output as clc
import sys
import pandas as pd

def stats(data, others=dict()):
    if not isinstance(data, pd.DataFrame):
        data = data.to_frame()
    stats = pd.concat([data.nunique(), data.dtypes, data.isnull().sum()], axis=1)
    stats.columns = ['Unique', 'Dtypes', 'NaN Count']
    for k,v in others.items():
        stats[k] = v(data)
    return stats

def page(data, wrap_cols=14):
    if not isinstance(data, pd.DataFrame):
        data = data.to_frame().T
    for i in range(wrap_cols, data.shape[1]+wrap_cols, wrap_cols):
        print ('Columns', i-wrap_cols, '-', min(i-1,data.shape[1]-1))
        display(data.iloc[:, i-wrap_cols:i])

C:\Users\spenc\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# tf.enable_eager_execution () # Don't turn on except for quick experiments. This is incompatible with the Estimator API.

In [3]:
tf.__version__

'1.7.0'

In [4]:
# Import dataset
D0 = pd.read_csv ('train_final.csv', index_col=0)
D0.head ()

,Y,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27
id,,,,,,,,,,,,,,,,,,,,,
1,0,1,0,0.107576,0,0.0,1,1,1,1,...,104,22902.0,1,0,18,0.042295,1,0,27,0.028250
2,0,1,0,0.142357,0,0.0,7,1,1,1,...,144,11400.0,1,0,8,0.021417,1,0,67,0.253574
3,0,1,0,0.492318,0,3.0,4205,1,1,3,...,112,4833.0,1,0,13,0.502212,1,1,35,0.373397
4,0,1,0,-0.053028,0,2.0,2,1,1,5,...,127,3250.0,1,1,8,0.000000,1,0,50,0.674254
5,0,1,0,0.730797,0,0.0,11,1,1,1,...,148,4000.0,1,1,5,0.787592,1,0,71,0.371157


In [5]:
# Count number unique values and NaNs
page (D0.nunique ().T)
page (D0.isnull ().sum ())

Columns 0 - 13


,Y,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13
0,2,9,12,49998,9,11,1880,9,9,322,23,43,9,10


Columns 14 - 27


,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27
0,16,10,310,9,83,8770,10,334,55,42562,9,14,83,41705


Columns 0 - 13


,Y,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13
0,0,0,0,0,0,1268,0,0,0,0,0,0,0,0


Columns 14 - 27


,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27
0,0,0,0,0,0,9828,0,0,0,0,0,0,0,0


In [6]:
# Fill NaNs to get new dataframe D1
D1 = D0.fillna (method='ffill')

# Confirm no NaNs
page (D1.isnull ().sum ())

Columns 0 - 13


,Y,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Columns 14 - 27


,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [7]:
# Compare value counts of filled column
page (D0.iloc[:,5].value_counts ())
page (D1.iloc[:,5].value_counts ())

Columns 0 - 10


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
F5,29110,8682,6516,3129,977,232,55,18,8,2,1


Columns 0 - 10


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
F5,29863,8904,6701,3202,1002,242,55,18,8,2,1


In [8]:
# Decide feature types (categorical/continuous) based on number of unique values
continuous_cols = [3, 6, 9, 16, 19, 21, 23, 27]
categorical_cols = list (set (range (1, 28)) - set (continuous_cols))
continuous_cols, categorical_cols

([3, 6, 9, 16, 19, 21, 23, 27],
 [1, 2, 4, 5, 7, 8, 10, 11, 12, 13, 14, 15, 17, 18, 20, 22, 24, 25, 26])

In [9]:
# View data types of categorical columns
page (D1.iloc[:,categorical_cols].dtypes)
page (D1['F5'].value_counts ())

Columns 0 - 13


,F1,F2,F4,F5,F7,F8,F10,F11,F12,F13,F14,F15,F17,F18
0,int64,int64,int64,float64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64


Columns 14 - 18


,F20,F22,F24,F25,F26
0,int64,int64,int64,int64,int64


Columns 0 - 10


,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0
F5,29863,8904,6701,3202,1002,242,55,18,8,2,1


In [10]:
# Convert F5 to int to meet tf.feature_column data type requirement
D2 = D1.copy ()
D2['F5'] = D1['F5'].astype ('int')
page (D2['F5'].value_counts ())

Columns 0 - 10


,0,1,2,3,4,5,6,7,8,9,10
F5,29863,8904,6701,3202,1002,242,55,18,8,2,1


In [ ]:
# Plot feature histograms

import math

# Set subplots layout
cols = 5
rows = math.ceil (D2.shape[1]/cols)
fig, ax = plt.subplots (rows, cols, figsize=(3*cols,3*rows), sharey=True)
plt.subplots_adjust (hspace=.3, wspace=.3)

# Plot histograms
for i, (a, col) in enumerate (zip (ax.flat, D2)):
    plt.sca (a)
    if D2[col].nunique () > 100:
        plt.hist (np.log1p (D2[col]), bins=50, color='r')
    else:
        plt.hist (D2[col], bins=50)
    plt.grid (axis='x')
    plt.title (col)

# Clip unused axes
for a in ax.flat[D2.shape[1]:]:
    plt.delaxes (a)

Conv1D shape analysis
---

`kernel size` refers to receptive field of kernel (the width of a kernel of 2 elements but power-of-two dilation).

Dilation changes the effective kernel size by:

$$\text{kernel_size} = \text{kernel_size} + (\text{kernel_size} - 1) * (\text{dilation} - 1)$$

| dilation rate | kernel_size |
| :-----------: | :---------: |
|       1       |      2      |
|       2       |      3      |
|       4       |      5      |
|       8       |      9      |
|      16       |     17      |
|      32       |     33      |

When $\text{dilation} = 2^{k-1}$ at layer $k$ and $\text{kernel_size} = 2$:

$$\text{kernel_size} = (2) + (2 - 1) * (2^{k-1} - 1) = 2^{k-1} + 1$$

`output width` refers to the width of the output with 'valid' padding


| kernel size | output width |
| :---------: | :----------: |
|     2       |        3     |
|     3       |        1     |


| kernel size | output width |
| :---------: | :----------: |
|     2       |        7     |
|     3       |        5     |
|     5       |        1     |


| kernel size | output width |
| :---------: | :----------: |
|     2       |       15     |
|     3       |       13     |
|     5       |        9     |
|     9       |        1     |


After $k$ layers, output dims is trimmed in width by $2^{k}-1$.

If $N = 2^{k}$, then output dims is reduced to $1$ after $k$ layers.

# EDIT: Actually just use valid padding
For $N=8$, this picture shows which outputs are actually used with valid padding. Many are wasted.
Same padding would make use of these unused outputs and is no worse than any other substitute estimate for the later samples in an input block.

`
[.][.][.][.][.][.][.][.]  input
[.] . [.] . [.] . [.]     layer 1 output
[.] .  .  . [.]           layer 2 output
[.]                       layer 3 output`

In [ ]:
tf.InteractiveSession ()

In [ ]:
# Scratch area: Just testing
a = tf.convert_to_tensor (np.r_['0,2',[0,0,0,4],[4,0,0,8],[8,0,0,12],[12,0,0,16]])
# tf.cast (a > 0, tf.double) * a
# a[1::2],a

b = tf.zeros ([4,4])
b = a
a *= 3
b += b
b.eval (), a.eval (), 


In [13]:
a = np.r_[0:4]
b = np.zeros ([4])
b = a[:]
a *= 2
b, a

(array([0, 2, 4, 6]), array([0, 2, 4, 6]))

In [31]:
# Set up model

Estimator = tf.estimator
batch_normalization = tf.layers.batch_normalization
Conv1D = tf.layers.Conv1D
input_layer = tf.feature_column.input_layer
import math
from functools import reduce


# Model settings
class PARAMS:
    k = 7
    width = 2**(k)
    batch_size = 128
    effective_batch_size = batch_size + width - 1
    kernel_size = 2
    depth = k
    

# Model definition
def model_fn (features, labels, mode, params):
    
    # (Step 0)
    # Identify run mode
    TRAIN = EVAL = PREDICT = False
    if mode == Estimator.ModeKeys.TRAIN:
        TRAIN = True
    elif mode == Estimator.ModeKeys.EVAL:
        EVAL = True
    elif mode == Estimator.ModeKeys.PREDICT:
        PREDICT = True
        
        
    # (Step 1)
    # Make input layer of all features, shape: (PARAMS.effective_batch_size, len (features))
    inputs = input_layer (features, params['feature_columns'])
    
    # Normalize inputs
    inputs = batch_normalization (inputs, training=TRAIN)
    
    
    # (Step 2)
    # Configure convolutional layers
    layers_config = [
        {
            'filters': 256,
            'kernel_size': PARAMS.kernel_size,
            'padding': 'same',
            'dilation_rate': 2**i,
            'name': 'Conv1D_{}'.format (i),
        }
        for i in range (PARAMS.depth)
    ]
    
    # Each layer module in layers[] contains a pair <projection, layer> to be applied together
    # in a residual block.
    # First layer module doesn't have a projection.
    layers = [
        [
            None,
            Conv1D (**layers_config[0])
        ]
    ]
    
    layers.extend ([
        [
            Conv1D(**{
                'filters': 64,
                'kernel_size': 1,
                'activation': tf.nn.leaky_relu,
                'name': 'Conv1D_{}_projection'.format (i)
            }),
            Conv1D(**config)
        ]
        for i, config in enumerate (layers_config[1:], 1)
    ])
    
    
    # Save layer configuration in the corresponding the layer
    for i, ((_, layer), config) in enumerate (zip (layers, layers_config)):
        layer.config = config
    
    # Configure final layer: 1x1 convolution with softmax
    final_layer = Conv1D(**{
        'filters': 1,
        'kernel_size': 1,
        'activation': tf.nn.softmax,
        'name': 'Softmax',
    })
    

    # Step (3)
    # Define residual connection block with batch normalization and relu activation
    def connect (inputs, layer_module, residual=True):
        print ('Connecting... layer {}'.format (layer_module[1].name))
        
        # Apply projection if exists
        projection, layer = layer_module
        if projection:
            result = projection (inputs)
        else:
            result = inputs
        
        # Apply main convolution
        result = layer (result)
        
        # Apply residual shortcut
        if residual:
            result += inputs
        
        # Apply batch normalize and activation
        result = batch_normalization (result, training=TRAIN)
        result = tf.nn.leaky_relu (result)
        
        return result
    
    # Define helper function to run layers
    def apply_to (inputs):
        result = reduce (
            connect,
            layers[1:],
            connect (inputs, layers[0], residual=False)
        )
        result = final_layer (result)
        return result
    
    
    # Step (4)
    # Obtain first-pass estimates on the output for each row of the inputs
    def first_pass (input_row):
        # Tile input row to full width (PARAMS.width)
        input_row = tf.reshape (input_row, [1, 1, -1])
        tiled_row = tf.tile (input_row, [1, PARAMS.width, 1])
        
        # Evaluate network on tiled row and average results
        result = apply_to (tiled_row)
        result = tf.reduce_mean (result)
        
        return result
    
    print ('inputs.shape:', inputs.shape)
    # Evaluate first_pass() on each row of the batch
    first_pass_estimates = tf.map_fn (first_pass, inputs, name='First_Pass')
    
    
    # Step (5)
    # Sort inputs based on first-pass estimated values
    _, sorted_indices = tf.nn.top_k (
        first_pass_estimates,
        k=PARAMS.effective_batch_size,
        name='Sorted_Indices'
    )
    sorted_inputs = tf.gather (inputs, sorted_indices, name='Sorted_Inputs')
    
    
    # Step (6)
    
    #######################################
    # TODO (4/5/17): Fix Step (6)
    #   - Don't do this iterative windowing
    #     The network does not care about width
    #     so long as its at least PARAMS.width
    #   - Incorporate self-iteration, use previous
    #     estimates as a feature and repeat a
    #     few times. This requires editing prior
    #     steps a bit. Perform loss minimization
    #     using self-predictions alpha-zero style.
    #######################################
    
    # Evaluate sorted inputs in blocks of <PARAMS.width> to produce <PARAMS.batch_size> predictions
    predictions = tf.Variable (tf.identity (first_pass_estimates, name='Predictions'))
    
    # Also keep track of how much predictions for a given input row varies
    variation = tf.Variable (tf.zeros ([PARAMS.effective_batch_size, 1], name='Same_Sample_Prediction_Pseudo_Variance'))
    
    for i in range (PARAMS.batch_size):
        # Prepare an input block
        input_block = inputs[i:i + PARAMS.width]
        input_block = tf.expand_dims (input_block, axis=0)
        
        # Apply layers
        result = apply_to (input_block)
        
        # Record difference from last prediction (add to previous, normalize later)
        tf.scatter_add (
            variation,
            indices=np.r_[i:i+PARAMS.width],
            updates=abs (predictions[i:i + PARAMS.width] - result)
        )
        
        # Average predictions with previous
        # Perform: predictions[i:i+PARAMS.width] = (predictions[i:i + PARAMS.width] + result) / 2
        tf.scatter_div (
            tf.scatter_add (
                predictions,
                indices=np.r_[i:i+PARAMS.width],
                updates=result
            ),
            indices=np.r_[i:i+PARAMS.width],
            updates=2,
        )
    
    # Normalize pseudo-variance
    variation[:PARAMS.width-1] /= np.r_[:PARAMS.width-1]
    variation[PARAMS.width-1:-PARAMS.width+1] /= PARAMS.width
    variation[-PARAMS.width+1:] /= np.r_[PARAMS.width-1:-1:-1]
    # Example:
    #   width, W = 4
    #   effective batch size, N = 8
    #
    #   Dots represent the number of predictions made for that input index.
    #   Indices i=[W-1:-W+1] have W predictions.
    #   Indices i=[:W-1] have i+1 predictions.
    #   Indices i=[-W+1:] have i+N predictions.
    #            .  .
    #         .  .  .  .
    #      .  .  .  .  .  .
    #   .  .  .  .  .  .  .  .
    #   0  1  2  3  4  5  6  7
    
    # Average prediction pseudo-variance
    avg_variation = tf.reduce_mean (variation, 'Average_Same_Sample_Prediction_Pseudo_Variance')
    tf.summary.scalar ('Average_Same_Sample_Prediction_Pseudo_Variance_Reported', avg_variation)
    
    
    # Step (7a)
    if PREDICT:
        output = {
            'probabilities': predictions
        }
        return Estimator.EstimatorSpec (mode=mode, predictions=output)


    # Step (8)
    # Define loss and metrics
    loss = tf.losses.softmax_cross_entropy(labels[sorted_indices], logits=predictions, name='Cross_Entropy')
    loss = .95 * loss + (1-.95) * avg_variation
    auc = tf.metrics.auc (labels=labels[sorted_indices], predictions=predictions)
    
    # Log metrics
    tf.summary.scalar ('AUC', auc)
    
    
    # Step (9)
    # Report metrics
    if EVAL:
        metrics = {
            'AUC': auc,
            'Avg. Prediction Variation': avg_variation
        }
        return Estimator.EstimatorSpec (mode=mode, loss=loss, eval_metric_ops=metrics)
    
    
    # Step (9)
    # Optimize loss
    if TRAIN:
        optimizer = tf.train.AdamOptimizer ()
        train_op = optimizer.minimize (loss, global_step=tf.train.get_global_step())
        return Estimator.EstimatorSpec (mode=mode, loss=loss, train_op=train_op)


In [15]:
# Set up input pipeline

categorical_column_with_hash_bucket = tf.feature_column.categorical_column_with_hash_bucket
embedding_column = tf.feature_column.embedding_column
numeric_column = tf.feature_column.numeric_column


# Training input function feeds from dataframe D1
train_fn = tf.estimator.inputs.pandas_input_fn (
    D2.iloc[:,1:],
    y=D2['Y'],
    batch_size=PARAMS.effective_batch_size,
    num_epochs=1,
    shuffle=True,
    queue_capacity=10*PARAMS.effective_batch_size,
    num_threads=8,
    target_column='Y'
)


# Preprocess features according to type:
# Make embeddings for categorical features
categorical = []

for col_idx in categorical_cols:
    name = D2.columns[col_idx]
    num_unique = D2.iloc[:,col_idx].nunique ()
    column = categorical_column_with_hash_bucket (
        key=name,
        hash_bucket_size=num_unique,
        dtype=tf.int32
    )
    embedded = embedding_column (column, math.ceil (math.log2 (num_unique)))
    categorical.append (embedded)
    
# No preprocessing for continuous features
continuous = [numeric_column (D2.columns[col_idx]) for col_idx in continuous_cols]

In [32]:
# Train model

# Instantiate from model_fn
model = Estimator.Estimator (
    model_fn,
    model_dir='model_checkpoint',
    params={
        'feature_columns': categorical + continuous
    }
)

# List of tensors to log
tensors_to_log = {
    'AUC': 'AUC'
}
hook = tf.train.LoggingTensorHook (tensors=tensors_to_log, every_n_iter=50)

# Train
model.train (input_fn=train_fn, hooks=[hook])

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model_checkpoint', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000025D7935F2B0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
inputs.shape: (?, 95)
Connecting... layer Conv1D_0
Connecting... layer Conv1D_1
Connecting... layer Conv1D_2
Connecting... layer Conv1D_3
Connecting... layer Conv1D_4
Connecting... layer Conv1D_5
Connecting... layer Conv1D_6


ValueError: initial_value must have a shape specified: Tensor("Predictions:0", shape=(?,), dtype=float32)